In [43]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt

## Weather / Forecast 

Weather forecasts that would have been available at prediction time. Sourced from the [European Centre for Medium-Range Weather Forecasts](https://codes.ecmwf.int/grib/param-db/?filter=grib2).

* `[latitude/longitude]`: The coordinates of the weather forecast.
* `origin_datetime`: The timestamp of when the forecast was generated.
* `hours_ahead`: The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
* `temperature` - The air temperature at 2 meters above ground in degrees Celsius.
* `dewpoint` - The dew point temperature at 2 meters above ground in degrees Celsius.
* `cloudcover_[low/mid/high/total]` - The percentage of the sky covered by clouds in the following altitude bands: 0-2 km, 2-6, 6+, and total.
* `10_metre_[u/v]_wind_component` - The [eastward/northward] component of wind speed measured 10 meters above surface in meters per second.
data_block_id
* `forecast_datetime`: The timestamp of the predicted weather. Generated from origin_datetime plus hours_ahead.
* `direct_solar_radiation`: The direct solar radiation reaching the surface on a plane perpendicular to the direction of the Sun accumulated during the preceding hour, in watt-hours per square meter.
* `surface_solar_radiation_downwards`: The solar radiation, both direct and diffuse, that reaches a horizontal plane at the surface of the Earth, in watt-hours per square meter.
* `snowfall`: Snowfall over the previous hour in units of meters of water equivalent.
* `total_precipitation`: The accumulated liquid, comprising rain and snow that falls on Earth's surface over the preceding hour, in units of meters.

In [59]:
df_weather_fc = pd.read_csv('../../input/forecast_weather.csv', parse_dates=['origin_datetime', 'forecast_datetime'])
df_weather_fc['forecast_datetime'] = df_weather_fc['forecast_datetime'].dt.tz_convert(None)
df_weather_fc.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424512 entries, 0 to 3424511
Data columns (total 18 columns):
 #   Column                             Non-Null Count    Dtype              
---  ------                             --------------    -----              
 0   latitude                           3424512 non-null  float64            
 1   longitude                          3424512 non-null  float64            
 2   origin_datetime                    3424512 non-null  datetime64[ns, UTC]
 3   hours_ahead                        3424512 non-null  int64              
 4   temperature                        3424512 non-null  float64            
 5   dewpoint                           3424512 non-null  float64            
 6   cloudcover_high                    3424512 non-null  float64            
 7   cloudcover_low                     3424512 non-null  float64            
 8   cloudcover_mid                     3424512 non-null  float64            
 9   cloudcover_total        

In [60]:
df_weather_fc.head()

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 00:00:00+00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0
1,57.6,22.2,2021-09-01 00:00:00+00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0
2,57.6,22.7,2021-09-01 00:00:00+00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0
3,57.6,23.2,2021-09-01 00:00:00+00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0
4,57.6,23.7,2021-09-01 00:00:00+00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0


In [61]:
location = pd.read_csv("../../input//county_lon_lats.csv").drop(columns = ["Unnamed: 0"])
for k in ['latitude', 'longitude'] :
    location[k] = (10*location[k]).astype(int)

In [62]:
location

,county,longitude,latitude
0,0,242,591
1,0,252,591
2,0,237,594
3,0,242,594
4,0,247,594
...,...,...,...
70,15,267,576
71,15,272,576
72,15,267,579
73,15,272,579


In [63]:
def get_county_loc(h:pd.DataFrame, location: pd.DataFrame=location) -> pd.DataFrame:
    """
    Maps Latitude and Longitude of the h dataframe to the county code
    """
    h = h.drop_duplicates().reset_index(drop=True)
    for k in ['latitude', 'longitude'] :
        h[k] = (10*h[k]).astype(int)
    h = pd.merge(h, location, how='left', on=['latitude', 'longitude'])
    h['county'] = h['county'].fillna(-1).astype(int)
    return h

In [64]:
df_weather_fc = get_county_loc(df_weather_fc, location)

In [65]:
df_weather_fc

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,county
0,576,217,2021-09-01 00:00:00+00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0,-1
1,576,222,2021-09-01 00:00:00+00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0,-1
2,576,227,2021-09-01 00:00:00+00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0,-1
3,576,232,2021-09-01 00:00:00+00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0,10
4,576,237,2021-09-01 00:00:00+00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 01:00:00,0.0,0.0,0.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424507,597,262,2023-05-30 00:00:00+00:00,48,10.150049,5.662744,0.061066,0.257339,0.004059,0.308594,6.890626,-3.258144,637,2023-06-01 00:00:00,0.0,0.0,0.0,0.0,5
3424508,597,267,2023-05-30 00:00:00+00:00,48,10.259790,5.922754,0.107880,0.895203,0.198700,0.906036,6.890626,-3.768032,637,2023-06-01 00:00:00,0.0,0.0,0.0,0.0,5
3424509,597,272,2023-05-30 00:00:00+00:00,48,10.467798,6.072534,0.000000,0.387466,0.448853,0.530518,6.214112,-4.700405,637,2023-06-01 00:00:00,0.0,0.0,0.0,0.0,-1
3424510,597,277,2023-05-30 00:00:00+00:00,48,10.694116,7.009058,0.000000,0.505936,0.997314,0.997498,5.810060,-4.338345,637,2023-06-01 00:00:00,0.0,0.0,0.0,0.0,-1


In [66]:
# Aggregate information over latitude/longitude
weather_agg = {'temperature' : ['min', 'mean', 'max', 'std'],
            'dewpoint' : ['min', 'mean', 'max', 'std'],
            'cloudcover_high' : ['min', 'mean', 'max', 'std'],
            'cloudcover_low' : ['min', 'mean', 'max', 'std'],
            'cloudcover_mid' : ['min', 'mean', 'max', 'std'],
            'cloudcover_total' : ['min', 'mean', 'max', 'std'],
            '10_metre_u_wind_component' : ['min', 'mean', 'max', 'std'],
            '10_metre_v_wind_component' : ['min', 'mean', 'max', 'std'],
            'direct_solar_radiation' : ['min', 'mean', 'max', 'std'],
            'surface_solar_radiation_downwards' : ['min', 'mean', 'max', 'std'],
            'snowfall' : ['min', 'mean', 'max', 'std'],
            'total_precipitation' : ['min', 'mean', 'max', 'std'],
        }

In [67]:
df_weather_fc = df_weather_fc.groupby(['county', 'forecast_datetime']).agg(weather_agg).reset_index()
df_weather_fc.columns = ['_'.join([xx for xx in x if len(xx)>0]) for x in df_weather_fc.columns]
df_weather_fc.columns = [x + '_f' if x not in ['county', 'forecast_datetime'] else x for x in df_weather_fc.columns]
df_weather_fc

,county,forecast_datetime,temperature_min_f,temperature_mean_f,temperature_max_f,temperature_std_f,dewpoint_min_f,dewpoint_mean_f,dewpoint_max_f,dewpoint_std_f,...,surface_solar_radiation_downwards_max_f,surface_solar_radiation_downwards_std_f,snowfall_min_f,snowfall_mean_f,snowfall_max_f,snowfall_std_f,total_precipitation_min_f,total_precipitation_mean_f,total_precipitation_max_f,total_precipitation_std_f
0,-1,2021-09-01 01:00:00,9.783228,13.162114,16.003564,2.008233,8.423364,11.012977,12.878687,1.149879,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
1,-1,2021-09-01 02:00:00,9.828760,13.108703,15.790552,1.958951,8.596460,10.802036,12.211328,0.944849,...,0.000000,0.000000,0.0,0.0,0.0,0.0,-4.470348e-07,1.308886e-07,0.000003,5.389440e-07
2,-1,2021-09-01 03:00:00,9.678247,13.114853,15.813745,1.773270,8.585840,10.714192,12.302271,0.931621,...,0.000000,0.000000,0.0,0.0,0.0,0.0,-1.788139e-07,4.019286e-07,0.000005,1.120368e-06
3,-1,2021-09-01 04:00:00,10.256006,13.275135,15.852930,1.454106,8.728418,10.698831,12.527612,0.946253,...,22.015305,6.190539,0.0,0.0,0.0,0.0,0.000000e+00,2.505811e-06,0.000026,6.217666e-06
4,-1,2021-09-01 05:00:00,11.425562,13.599136,15.797754,1.048089,8.849512,10.684499,12.416040,0.955025,...,121.433098,22.037336,0.0,0.0,0.0,0.0,0.000000e+00,1.420846e-06,0.000024,4.435588e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244987,15,2023-05-31 20:00:00,11.981592,12.470825,13.443750,0.593861,6.649805,6.987988,7.293115,0.305610,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
244988,15,2023-05-31 21:00:00,10.862939,11.484033,12.518091,0.668929,5.690820,6.748584,7.131494,0.600507,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
244989,15,2023-05-31 22:00:00,9.960474,10.464868,11.626001,0.704654,4.415186,5.860254,6.782617,0.914607,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
244990,15,2023-05-31 23:00:00,9.068506,9.544800,10.520532,0.595610,3.746973,4.782935,5.657861,0.769510,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00


In [68]:
df_weather_fc.columns

Index(['county', 'forecast_datetime', 'temperature_min_f',
       'temperature_mean_f', 'temperature_max_f', 'temperature_std_f',
       'dewpoint_min_f', 'dewpoint_mean_f', 'dewpoint_max_f', 'dewpoint_std_f',
       'cloudcover_high_min_f', 'cloudcover_high_mean_f',
       'cloudcover_high_max_f', 'cloudcover_high_std_f',
       'cloudcover_low_min_f', 'cloudcover_low_mean_f', 'cloudcover_low_max_f',
       'cloudcover_low_std_f', 'cloudcover_mid_min_f', 'cloudcover_mid_mean_f',
       'cloudcover_mid_max_f', 'cloudcover_mid_std_f',
       'cloudcover_total_min_f', 'cloudcover_total_mean_f',
       'cloudcover_total_max_f', 'cloudcover_total_std_f',
       '10_metre_u_wind_component_min_f', '10_metre_u_wind_component_mean_f',
       '10_metre_u_wind_component_max_f', '10_metre_u_wind_component_std_f',
       '10_metre_v_wind_component_min_f', '10_metre_v_wind_component_mean_f',
       '10_metre_v_wind_component_max_f', '10_metre_v_wind_component_std_f',
       'direct_solar_radiation_

In [69]:
df_weather_fc["forecast_datetime"]

0        2021-09-01 01:00:00
1        2021-09-01 02:00:00
2        2021-09-01 03:00:00
3        2021-09-01 04:00:00
4        2021-09-01 05:00:00
                 ...        
244987   2023-05-31 20:00:00
244988   2023-05-31 21:00:00
244989   2023-05-31 22:00:00
244990   2023-05-31 23:00:00
244991   2023-06-01 00:00:00
Name: forecast_datetime, Length: 244992, dtype: datetime64[ns]

In [70]:
# df_weather_fc["forecast_datetime"] = pd.to_datetime(df_weather_fc)

# df_weather_fc["forecast_datetime"]

In [71]:
from enefit_challenge.utils.dataset import load_enefit_training_data

df = load_enefit_training_data()

In [72]:
df = df.merge(
    df_weather_fc.rename(columns = {'forecast_datetime' : 'datetime'}),
    how='left',
    on=['county', 'datetime'],
)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,surface_solar_radiation_downwards_max_f,surface_solar_radiation_downwards_std_f,snowfall_min_f,snowfall_mean_f,snowfall_max_f,snowfall_std_f,total_precipitation_min_f,total_precipitation_mean_f,total_precipitation_max_f,total_precipitation_std_f
0,0,0,1,0.713,0,2021-09-01,0,0,0,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,0,2,7.620,1,2021-09-01,0,89,44,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,0,2,0.000,0,2021-09-01,0,88,44,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,0,1,21.099,1,2021-09-01,0,87,43,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,0,1,0.000,0,2021-09-01,0,86,43,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
for k in df :
    if k.endswith('_f') :
        df[k] = df[k].ffill().bfill()

In [75]:
df.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,surface_solar_radiation_downwards_max_f,surface_solar_radiation_downwards_std_f,snowfall_min_f,snowfall_mean_f,snowfall_max_f,snowfall_std_f,total_precipitation_min_f,total_precipitation_mean_f,total_precipitation_max_f,total_precipitation_std_f
0,0,0,1,0.713,0,2021-09-01,0,0,0,2021-09-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11,0,2,7.620,1,2021-09-01,0,89,44,2021-09-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11,0,2,0.000,0,2021-09-01,0,88,44,2021-09-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11,0,1,21.099,1,2021-09-01,0,87,43,2021-09-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11,0,1,0.000,0,2021-09-01,0,86,43,2021-09-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
df.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id', 'date',
       'time', 'year', 'datediff_in_days', 'hour', 'hour_sine', 'hour_cosine',
       'dayofweek', 'dayofweek_sine', 'dayofweek_cosine', 'week', 'week_sine',
       'week_cosine', 'month', 'month_sine', 'month_cosine',
       'target_2_days_ago', 'eic_count', 'installed_capacity', 'euros_per_mwh',
       'lowest_price_per_mwh', 'highest_price_per_mwh', 'temperature_min_f',
       'temperature_mean_f', 'temperature_max_f', 'temperature_std_f',
       'dewpoint_min_f', 'dewpoint_mean_f', 'dewpoint_max_f', 'dewpoint_std_f',
       'cloudcover_high_min_f', 'cloudcover_high_mean_f',
       'cloudcover_high_max_f', 'cloudcover_high_std_f',
       'cloudcover_low_min_f', 'cloudcover_low_mean_f', 'cloudcover_low_max_f',
       'cloudcover_low_std_f', 'cloudcover_mid_min_f', 'cloudcover_mid_mean_f',
       'cloudcover_mid_max_f', 'cloudcover_mid